<a href="https://colab.research.google.com/github/eshikapathak/Personalized-Federated-Learning/blob/main/Basic/FedAvg_and_FedSGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FedAvg

## Imports


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, TensorDataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random
import copy
from torchsummary import summary

## Data loaders

In [2]:
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [3]:
def iid_loader(dataset, num_clients=100, batch_size=10):
    total = len(dataset)
    indices = torch.randperm(total).tolist()
    data_per_client = total // num_clients
    client_indices = [indices[i * data_per_client:(i + 1) * data_per_client] for i in range(num_clients)]

    client_loaders = [
        DataLoader(Subset(dataset, indices), batch_size=batch_size, shuffle=True)
        for indices in client_indices
    ]
    return client_loaders

In [4]:
def non_iid_loader(dataset, num_clients=100, batch_size=10, classes_per_user=2, high_prob=0.6, low_prob=0.4):
    # Helper function to get number of classes and labels
    def get_num_classes_samples(data):
        if isinstance(data.targets, list):
            labels = np.array(data.targets)
        else:
            labels = data.targets.numpy()
        classes, num_samples = np.unique(labels, return_counts=True)
        return len(classes), num_samples, labels

    # Helper function to generate data split based on class partitions
    def gen_data_split(num_samples, labels, class_partitions):
        data_class_idx = {i: np.where(labels == i)[0] for i in range(num_classes)}
        for idx_list in data_class_idx.values():
            random.shuffle(idx_list)

        user_data_idx = [[] for _ in range(num_clients)]
        for usr_i in range(num_clients):
            for c, p in zip(class_partitions['class'][usr_i], class_partitions['prob'][usr_i]):
                end_idx = int(p * num_samples[c])
                user_data_idx[usr_i].extend(data_class_idx[c][:end_idx])
                data_class_idx[c] = data_class_idx[c][end_idx:]
        return user_data_idx

    num_classes, num_samples, labels = get_num_classes_samples(dataset)
    count_per_class = (classes_per_user * num_clients) // num_classes

    # Generating class partitions
    class_dict = {i: {'prob': np.random.uniform(low_prob, high_prob, size=count_per_class).tolist()} for i in range(num_classes)}
    for probs in class_dict.values():
        total = sum(probs['prob'])
        probs['prob'] = [p / total for p in probs['prob']]

    # Assign classes and probabilities to each client
    class_partitions = {'class': [], 'prob': []}
    available_classes = list(range(num_classes)) * count_per_class
    random.shuffle(available_classes)
    for _ in range(num_clients):
        client_classes = random.sample(available_classes, classes_per_user)
        for c in client_classes:
            available_classes.remove(c)
        client_probs = [class_dict[c]['prob'].pop() for c in client_classes]
        class_partitions['class'].append(client_classes)
        class_partitions['prob'].append(client_probs)

    # Generating data splits
    user_data_idx = gen_data_split(num_samples, labels, class_partitions)

    # Creating data loaders
    client_data_loaders = []
    for indices in user_data_idx:
        subset = Subset(dataset, indices)
        loader = DataLoader(subset, batch_size=batch_size, shuffle=True)
        client_data_loaders.append(loader)

    return client_data_loaders

### Models

In [5]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(28*28, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 10),
        )

    def forward(self, x):
        x = x.flatten(1)
        x = self.nn(x)
        return x

class CNN(nn.Module):
    def __init__(self, in_channels=3, n_kernels=16, out_dim=10):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, n_kernels, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(n_kernels, 2 * n_kernels, 5)
        self.fc1 = nn.Linear(2 * n_kernels * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, out_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Train fxns

In [45]:
def train(model, client_train_loader, epochs, optimizer, criterion):
    model.train()
    for _ in range(epochs):
        for _, (data, target) in enumerate(client_train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model

def evaluate(model, test_loader):
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax( dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    acc = correct / total
    return acc

In [46]:
def running_model_average(current, next, scale):
    if current == None:
        current = next
        for key in current:
            current[key] = current[key]*scale
    else:
        for key in current:
            current[key] = current[key] + next[key]*scale
    return current

In [54]:
def experiments(global_model, num_clients, client_frac, epochs, lr, train_loader, test_loader, max_rounds, criterion):
    val_accuracy = []
    train_accuracy = []
    for t in range(max_rounds):
        #print("starting round {}".format(t))

        clients = np.random.choice(np.arange(num_clients), int(num_clients*client_frac), replace = False)
        #print("clients: ", clients)

        global_model.eval()
        global_model = global_model.to(device)
        running_avg = None
        for i, c in enumerate(clients):
            #print("round {}, starting client {}/{}, id: {}".format(t, i+1,num_clients*client_frac, c))
            local_model = copy.deepcopy(global_model).to(device)
            optimizer = torch.optim.SGD(local_model.parameters(), lr = lr)
            local_model = train(local_model, train_loader[c], epochs=epochs, optimizer=optimizer, criterion = criterion)
            running_avg = running_model_average(running_avg, local_model.state_dict(), 1/(num_clients*client_frac))

        global_model.load_state_dict(running_avg)
        val_acc = evaluate(global_model, test_loader)
        print("round {}, validation acc: {}".format(t, val_acc))
        val_accuracy.append(val_acc)

        # train_acc = evaluate(global_model, train_loader)
        # train_accuracy.append(train_acc)

    return np.array(val_accuracy)

In [48]:
def get_final_loaders(train_data, num_clients, iid = 0, batch_size = 64, classes_per_user = 2):
  if iid == 1: # want iid
    train_loader = iid_loader(train_data, num_clients, batch_size)
  else: # want non iid
    train_loader = non_iid_loader(train_data, num_clients, batch_size, classes_per_user=classes_per_user)
  test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
  return train_loader, test_loader

In [50]:
# cnn = CNN(n_kernels = 11).to(device)
# summary(cnn,(3,32,32))

In [38]:
# # Checking how many samples are allocated to each client

# import numpy as np
# import torch
# from torch.utils.data import DataLoader, Subset
# import random

# # Call the function to get the list of DataLoader objects
# client_data_loaders = non_iid_loader(train_data, num_clients=10, batch_size=64, classes_per_user=2)

# # Iterate through each DataLoader in the list and print the number of samples it contains
# for idx, loader in enumerate(client_data_loaders):
#     # Each loader's dataset attribute is a Subset of the original dataset
#     print(f"Loader {idx+1}: {len(loader.dataset)} samples")

# Experiments

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading and transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [52]:
batch_size = 64
num_classes = 10
client_frac = 1
max_rounds = 1000
criterion = nn.CrossEntropyLoss()

## EXPT1: 10 clients, 5 internal epochs

In [ ]:
iid = 0 # non-iid
num_clients = 10
epochs = 5

cnn = CNN(n_kernels=11) # cnn2
cnn_niid_10_5 = copy.deepcopy(cnn)

train_loader, test_loader = get_final_loaders(train_data, num_clients, iid = iid, batch_size = batch_size, classes_per_user = 2)
acc_cnn_niid_10_5_val = experiments(cnn_niid_10_5, num_clients = num_clients, client_frac = client_frac, epochs= epochs, lr = 0.05, train_loader= train_loader, test_loader = test_loader, max_rounds=max_rounds, criterion=criterion)
# Save the accuracy in an NPZ file
np.savez("acc_cnn_niid_10_5.npz", val_accuracy=acc_cnn_niid_10_5_val) # train_accuracy=acc_cnn_niid_10_5_train,
print("Done")

round 0, validation acc: 0.1001
round 1, validation acc: 0.1262
round 2, validation acc: 0.2696
round 3, validation acc: 0.321


## EXPT2: 10 clients, 20 internal epochs

In [ ]:
iid = 0 # non-iid
num_clients = 10
epochs = 20

cnn = CNN(n_kernels=11) # cnn2
cnn_niid_10_20 = copy.deepcopy(cnn)

train_loader, test_loader = get_final_loaders(train_data, num_clients, iid = iid, batch_size = batch_size, classes_per_user = 2)
acc_cnn_niid_10_20_val = experiments(cnn_niid_10_20, num_clients = num_clients, client_frac = client_frac, epochs= epochs, lr = 0.05, train_loader= train_loader, test_loader = test_loader, max_rounds=max_rounds, criterion=criterion)
# Save the accuracy in an NPZ file
np.savez("acc_cnn_niid_10_20.npz", val_accuracy=acc_cnn_niid_10_20_val) # train_accuracy=acc_cnn_niid_10_5_train,
print("Done")

## EXPT3: 20 clients, 5 internal epochs

In [ ]:
iid = 0 # non-iid
num_clients = 20
epochs = 5

cnn = CNN(n_kernels=11) # cnn2
cnn_niid_20_5 = copy.deepcopy(cnn)

train_loader, test_loader = get_final_loaders(train_data, num_clients, iid = iid, batch_size = batch_size, classes_per_user = 2)
acc_cnn_niid_20_5_val = experiments(cnn_niid_20_5, num_clients = num_clients, client_frac = client_frac, epochs= epochs, lr = 0.05, train_loader= train_loader, test_loader = test_loader, max_rounds=max_rounds, criterion=criterion)
# Save the accuracy in an NPZ file
np.savez("acc_cnn_niid_20_5.npz", val_accuracy=acc_cnn_niid_20_5_val) # train_accuracy=acc_cnn_niid_10_5_train,
print("Done")

## EXPT4: 20 clients, 20 internal epochs

In [ ]:
iid = 0 # non-iid
num_clients = 20
epochs = 20

cnn = CNN(n_kernels=11) # cnn2
cnn_niid_20_20 = copy.deepcopy(cnn)

train_loader, test_loader = get_final_loaders(train_data, num_clients, iid = iid, batch_size = batch_size, classes_per_user = 2)
acc_cnn_niid_20_20_val = experiments(cnn_niid_20_20, num_clients = num_clients, client_frac = client_frac, epochs= epochs, lr = 0.05, train_loader= train_loader, test_loader = test_loader, max_rounds=max_rounds, criterion=criterion)
# Save the accuracy in an NPZ file
np.savez("acc_cnn_niid_20_20.npz", val_accuracy=acc_cnn_niid_20_20_val) # train_accuracy=acc_cnn_niid_10_5_train,
print("Done")